In [ ]:
import os

import pandas as pd
import numpy as np

from math import pi

In [ ]:
cwd = os.getcwd()
join = os.path.join
norm = os.path.normpath

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = [15, 9]
plt.rcParams['font.size'] = 14

# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [ ]:
path = norm(join(os.getcwd(), '../../validation/MFRTRT'))
df_raw = pd.read_csv(join(path, 'MFRTRT_loads.csv'), index_col=[0], parse_dates=True)
df_raw.head(1)

In [ ]:
ks = 2.7
H = 76.2
rho_cp_s = 2500 * 850
alpha = ks / rho_cp_s
ts = H**2 / (9*alpha)
R_b = 0.194

In [ ]:
df = df_raw.copy(deep=True)

In [ ]:
df['Mean HT [W]'] = df[['Inst. HT [1] [W]', 'Inst. HT [2] [W]']].mean(axis=1)
df['Mean EWT [C]'] = df[['Inlet 1 [C]', 'Inlet 2 [C]']].mean(axis=1)
df['EWT - Tg [C]'] = df['Mean EWT [C]'] - df['Mean EWT [C]'].iloc[0]
df['q [W/m]'] = df['Mean HT [W]'] / H
df['delta t'] = df.index.to_series().diff().dt.seconds.div(1, fill_value=0)
df['d/dt (EWT - Tg) [C/s]'] = df['EWT - Tg [C]'] / df['delta t']
df['t [s]'] = df['delta t'].cumsum()
df['t/ts'] = df['t [s]'] / ts

In [ ]:
# df = df.resample('1T').mean()

In [ ]:
df[['EWT - Tg [C]', 'd/dt (EWT - Tg) [C/s]']].loc['2013-12-20 12:00:00' : '2013-12-20 12:30:00'].plot(marker='x')

In [ ]:
df.dropna(inplace=True, subset=['d/dt (EWT - Tg) [C/s]'])

In [ ]:
df['EWT - Tg [C]'].loc[df['d/dt (EWT - Tg) [C/s]'] < 0.025] = 0

In [ ]:
df[['EWT - Tg [C]', 'd/dt (EWT - Tg) [C/s]']].loc['2013-12-20 12:00:00' : '2013-12-20 12:30:00'].plot(marker='x')

In [ ]:
df[['q [W/m]']].loc['2013-12-20 12:00:00' : '2013-12-20 12:30:00'].plot()

In [ ]:
df['g_b [-]'] = df['EWT - Tg [C]'] / (df['q [W/m]'] * R_b)

In [ ]:
df.loc['2013-12-20 12:00:00' : '2013-12-22 12:00:00'].plot(x='t/ts', y='g_b [-]', logx=True)

In [ ]:
plt.semilogx(np.logspace(-7.6, -3.8, num=75), np.ones([75,]), marker='x')

In [ ]:
times = np.around(np.logspace(-7.6, -3.6, num=100) * ts)
times

In [ ]:
rolled = np.roll(times, shift=-1)
rolled[-1] = 1.4e5
rolled

In [ ]:
def calc_g_vals(in_times):
    ret_times = []
    ret_g = []
    for idx, low in enumerate(in_times):
        high = rolled[idx]
        val = df.loc[(df['t [s]'] > low) & (df['t [s]'] <= high)]['g_b [-]'].mean()
        if not np.isnan(val):
            ret_times.append(np.mean([low, high]))
            ret_g.append(val)
    return np.array(ret_times), np.array(ret_g)

In [ ]:
t, g_b = calc_g_vals(times)

In [ ]:
t_ts = t / ts

In [ ]:
t_ts_raw = df.loc['2013-12-20 12:00:00' : '2013-12-22 12:00:00']['t/ts'].values.tolist()
g_b_raw = df.loc['2013-12-20 12:00:00' : '2013-12-22 12:00:00']['g_b [-]'].values.tolist()
plt.semilogx(t_ts_raw, g_b_raw, label='Raw')
plt.semilogx(t_ts, g_b, marker='x', label='Sim Data')
plt.xlabel('t/ts')
plt.ylabel('g_b')
plt.xlim(1e-8, 1e-3)
plt.legend()
plt.show()

In [ ]:
out_data = np.array([np.log(t_ts), g_b])

In [ ]:
np.savetxt('exp_ewt_g_functions.csv', np.transpose(out_data), delimiter=',')